# Chapter 13: Naive Bayes

In [17]:
from __future__ import division
from collections import defaultdict
import glob, math, re

In [21]:
%%capture
%run 'machine_learning.ipynb'

ZeroDivisionError: integer division or modulo by zero

## Naive Bayes Implementation

In [4]:
def tokenize(message):
    message = message.lower()
    all_words = re.findall("[a-z0-9']", message) # words can contain letters, numbers, and apostrophes 
    return set(all_words)

In [10]:
def count_words(training_set):
    """ training set elements are pairs (message, is_spam)"""
    counts = defaultdict(lambda: [0,0])
    for message, is_spam in training_set:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1] +=1  # could also write counts[words][!is_spam] but this is more clear
    return counts

In [11]:
def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    """convert word counts into list of triplets (word, p(w|spam), p(w|~spam))"""
    return [(word, 
             (spam_count + k)/(total_spams + 2 * k),
             (non_spam_count + k)/(total_non_spams + 2 * k))
            for w, (spam_count, non_spam_count) in counts.iteritems()]

In [15]:
def spam_probability(word_probs, message):
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0
    
    for word, prob_if_spam, prob_if_not_spam in word_probs:
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)
    
    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

In [16]:
class NaiveBayesClassifier:
    
    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []
    
    def train(self, training_set):
        num_spams = len([b for _, is_spam in training_set if is_spam])
        num_non_spams = len(training_set) - num_spams
        
        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts, num_spams, num_non_spams, self.k)
    
    def classify(self, message):
        return spam_probability(self.word_probs, message)

### Testing Implementation

The following tests use the SpamAssassin public corpus (http://spamassassin.apache.org/publiccorpus/).

In [18]:
path = r"./spam/*/*"

In [20]:
def get_subject_data(path):
    data = []
    for fn in glob.glob(path):
        is_spam = "ham" not in fn
        with open(fn, 'r') as f:
            for line in f:
                if line.startswith("Subject:"):
                    subject = re.sub(r"^Subject: ", "", line).strip()
                    data.append((subject, is_spam))
    return data